In [0]:
import os
from google.colab import drive
import sys

In [2]:
### ONLY IF USING GOOGLE COLAB 
drive.mount('/content/drive')

print(os.getcwd())
os.chdir("/content/drive/My Drive/COMS4995_FinalProject")
os.getcwd()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content


'/content/drive/My Drive/COMS4995_FinalProject'

In [0]:
sys.argv = ["--cell_1=E116", # placeholder i think because -h messes it up
            "--cell_1=E116", 
            "--cell_2=E123", 
            "--model_name=aux_siamese", 
            "--epochs=20", 
            "--lr=0.001", 
            "--data_root=data/", 
            "--save_root=Results/", 
            "--attentionfilename=E116_E123_compareSC.csv", 
            "--n_bins=1000", 
            "--n_hms=5", 
            "--save_attention_maps"]


In [0]:
import warnings
warnings.filterwarnings("ignore")
import argparse
import json
import matplotlib
import matplotlib.pyplot as plt
import math
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import cuda
import sys, os
import random
import numpy as np
from sklearn import metrics
import models as Model
from SiameseLoss import ContrastiveLoss
import evaluate
import data_v1
import gc
import csv


from __future__ import print_function
import argparse
import torch
import csv
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np


In [0]:
parser = argparse.ArgumentParser(description='DeepDiff')
parser.add_argument('--lr', type=float, default=0.0001, help='initial learning rate')
parser.add_argument('--model_name', type=str, default='raw_d', help='DeepDiff variation')
parser.add_argument('--clip', type=float, default=1,help='gradient clipping')
parser.add_argument('--epochs', type=int, default=90, help='upper epoch limit')
parser.add_argument('--batch_size', type=int, default=10, help='')
parser.add_argument('--dropout', type=float, default=0.5, help='dropout applied to layers (0 = no dropout) if n_layers LSTM > 1')
parser.add_argument('--cell_1', type=str, default='Cell1', help='cell type 1')
parser.add_argument('--cell_2', type=str, default='Cell2', help='cell type 2')
parser.add_argument('--save_root', type=str, default='./Results/', help='where to save')
parser.add_argument('--data_root', type=str, default='./data/', help='data location')
parser.add_argument('--gpuid', type=int, default=0, help='CUDA gpu')
parser.add_argument('--gpu', type=int, default=0, help='CUDA gpu')
parser.add_argument('--n_hms', type=int, default=5, help='number of histone modifications')
parser.add_argument('--n_bins', type=int, default=200, help='number of bins')
parser.add_argument('--bin_rnn_size', type=int, default=32, help='bin rnn size')
parser.add_argument('--num_layers', type=int, default=1, help='number of layers')
parser.add_argument('--unidirectional', action='store_true', help='bidirectional/undirectional LSTM')
parser.add_argument('--save_attention_maps',action='store_true', help='set to save validation beta attention maps')
parser.add_argument('--attentionfilename', type=str, default='beta_attention.txt', help='where to save attnetion maps')
parser.add_argument('--test_on_saved_model',action='store_true', help='only test on saved model')
args = parser.parse_args()

torch.manual_seed(1)

model_name = ''
model_name += (args.cell_1)+('_')+(args.cell_2)+('_')

model_name+=args.model_name




args.bidirectional=not args.unidirectional

print('the model name: ',model_name)
args.data_root+=''
args.save_root+=''
args.dataset=args.cell_1+('_')+args.cell_2
args.data_root = os.path.join(args.data_root)
print('loading data from:  ',args.data_root)
args.save_root = os.path.join(args.save_root,args.dataset)
print('saving results in  from: ',args.save_root)
model_dir = os.path.join(args.save_root,model_name)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
attentionmapfile=model_dir+'/'+args.attentionfilename
print('==>processing data')


# return processed data before loading
# train_inpt, valid_inpt, test_inpt = data_v1.load_data(args)
train_inpt, valid_inpt, test_inpt = load_data(args)







CON=False
AUX=False
print('==>building model')
if(args.model_name=='raw_d'):
    model = Model.raw_d(args)
    # model = raw_d(args)

elif(args.model_name=='raw_c'):
    model = Model.raw_c(args)
    # model = raw_c(args)

elif(args.model_name=='raw'):
    model = Model.raw(args)
    # model = raw(args)

elif(args.model_name=='aux'):
    args.shared=False
    
    model = Model.aux(args)
    # model = aux(args)

    AUX=True
    args.gamma=0.0
elif(args.model_name=='raw_aux'):
    args.shared=False

    model = Model.raw_aux(args)
    # model = raw_aux(args)
    
    AUX=True
    args.gamma=0.0
elif(args.model_name=='aux_siamese'):
    CON=True
    args.shared=True
    
    model = Model.aux_siamese(args)
    # model = aux_siamese(args)
    
    AUX=True
    args.gamma=4.0
elif(args.model_name=='raw_aux_siamese'):
    CON=True
    args.shared=True

    model = Model.raw_aux_siamese(args)
    # model = raw_aux_siamese(args)

    AUX=True
    args.gamma=4.0
else:
    sys.exit("invalid model name")


if torch.cuda.device_count() >= 1:
    torch.cuda.manual_seed_all(1)
    dtype = torch.cuda.FloatTensor
    cuda.set_device(args.gpuid)
    model.type(dtype)
    print('Using GPU '+str(args.gpuid))
else:
    print("No GPU Available")
    dtype = torch.FloatTensor

## PRINTING MODEL USES SO MUCH SPACE WHY
#print(model)


if(args.test_on_saved_model==False):
    print("==>initializing a new model")
    for p in model.parameters():
        p.data.uniform_(-0.1,0.1)

DiffLoss = nn.MSELoss(size_average=True).type(dtype)
AuxLoss = nn.MSELoss(size_average=True).type(dtype)
ConLoss = ContrastiveLoss().type(dtype)

optimizer = optim.Adam(model.parameters(), lr = args.lr)
#optimizer = optim.SGD(model.parameters(), lr = args.lr, momentum=args.momentum)
def train(TrainData):
    model.train()
    # initialize attention
    diff_targets = torch.zeros(TrainData.dataset.__len__(),1)
    diff_predictions = torch.zeros(diff_targets.size(0),1)
    if(args.model_name=='raw_d'):
        all_attention_bin=torch.zeros(TrainData.dataset.__len__(),(args.n_hms*args.n_bins))
        all_attention_hm=torch.zeros(TrainData.dataset.__len__(),args.n_hms)
    elif(args.model_name=='raw_c'):
        all_attention_bin=torch.zeros(TrainData.dataset.__len__(),(2*args.n_hms*args.n_bins))
        all_attention_hm=torch.zeros(TrainData.dataset.__len__(),2*args.n_hms)
    elif(args.model_name=='raw'):
        all_attention_bin=torch.zeros(TrainData.dataset.__len__(),(3*args.n_hms*args.n_bins))
        all_attention_hm=torch.zeros(TrainData.dataset.__len__(),3*args.n_hms)

    elif(args.model_name=='aux' or args.model_name=='aux_siamese'):
        all_attention_bin=torch.zeros(TrainData.dataset.__len__(),(2*args.n_hms*args.n_bins))
        all_attention_hm=torch.zeros(TrainData.dataset.__len__(),2*args.n_hms)

    elif(args.model_name=='raw_aux' or args.model_name=='raw_aux_siamese'):
        all_attention_bin=torch.zeros(TrainData.dataset.__len__(),(3*args.n_hms*args.n_bins))
        all_attention_hm=torch.zeros(TrainData.dataset.__len__(),5*args.n_hms)

    else:
        all_attention_bin=torch.zeros(TrainData.dataset.__len__(),(args.n_hms*args.n_bins))
        all_attention_hm=torch.zeros(TrainData.dataset.__len__(),args.n_hms)

    num_batches = int(math.ceil(TrainData.dataset.__len__()/float(args.batch_size)))
    all_gene_ids=[None]*TrainData.dataset.__len__()
    per_epoch_loss = 0
    for idx, Sample in enumerate(TrainData):
        if(idx%100==0):
            print('TRAINING ON BATCH:',idx)
        start,end = (idx*args.batch_size), min((idx*args.batch_size)+args.batch_size, TrainData.dataset.__len__())
        optimizer.zero_grad()
        # get HM profiles
        inputs_1 = Sample['X_A']
        inputs_2 = Sample['X_B']


        # get targets: both differential and cell specific expression
        batch_diff_targets=(Sample['diff']).float().unsqueeze(1)
        batch_diff_targets_c1=(Sample['abs_A']).float().unsqueeze(1)
        batch_diff_targets_c2=(Sample['abs_B']).float().unsqueeze(1)
        diff_targets[start:end,0] = batch_diff_targets[:,0]

        if(CON==True):
            # get labels for contrastive loss
            batch_contrastive_targets =[]
            for label in batch_diff_targets:
                if(label<=-2.0):
                    batch_contrastive_targets.append(1)
                elif(label>=2.0):
                    batch_contrastive_targets.append(1)
                else:
                    batch_contrastive_targets.append(0)
            batch_contrastive_targets=torch.Tensor(batch_contrastive_targets)


        all_gene_ids[start:end]=Sample['geneID']
        batch_size = inputs_1.size(0)

        if(AUX==False):
            # for raw models: raw_d, raw_c, raw
            batch_diff_predictions,batch_beta,batch_alpha = model(inputs_1.type(dtype),inputs_2.type(dtype))

            all_attention_bin[start:end]=batch_alpha.data
            all_attention_hm[start:end]=batch_beta.data
            loss = DiffLoss(batch_diff_predictions,batch_diff_targets.type(dtype))
        elif(CON==False):
            # for aux models
            batch_diff_predictions,batch_beta,batch_alpha,batch_diff_predictions_c1,batch_diff_predictions_c2 = model(inputs_1.type(dtype),inputs_2.type(dtype))
            all_attention_bin[start:end]=batch_alpha.data
            all_attention_hm[start:end]=batch_beta.data
            loss = DiffLoss(batch_diff_predictions,batch_diff_targets.type(dtype))
            loss+=AuxLoss(batch_diff_predictions_c1,batch_diff_targets_c1.type(dtype))
            loss+=AuxLoss(batch_diff_predictions_c2,batch_diff_targets_c2.type(dtype))
        else:
            # for aux and siamese models
            batch_diff_predictions,batch_beta,batch_alpha,embedding_1,embedding_2,batch_diff_predictions_c1,batch_diff_predictions_c2 = model(inputs_1.type(dtype),inputs_2.type(dtype))

            all_attention_bin[start:end]=batch_alpha.data
            all_attention_hm[start:end]=batch_beta.data
            loss = DiffLoss(batch_diff_predictions,batch_diff_targets.type(dtype))
            loss+=AuxLoss(batch_diff_predictions_c1,batch_diff_targets_c1.type(dtype))
            loss+=AuxLoss(batch_diff_predictions_c2,batch_diff_targets_c2.type(dtype))
            loss+=args.gamma*ConLoss(embedding_1,embedding_2,batch_contrastive_targets.type(dtype))

        diff_predictions[start:end] = batch_diff_predictions.data.cpu()
        per_epoch_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm(model.parameters(), args.clip)
        optimizer.step()
    per_epoch_loss=per_epoch_loss/num_batches
    return diff_predictions,diff_targets,all_attention_bin,all_attention_hm,per_epoch_loss,all_gene_ids



def test(ValidData):
    model.eval()

    diff_targets = torch.zeros(ValidData.dataset.__len__(),1)
    diff_predictions = torch.zeros(diff_targets.size(0),1)
    if(args.model_name=='raw_d'):
        all_attention_bin=torch.zeros(ValidData.dataset.__len__(),(args.n_hms*args.n_bins))
        all_attention_hm=torch.zeros(ValidData.dataset.__len__(),args.n_hms)
    elif(args.model_name=='raw_c'):
        all_attention_bin=torch.zeros(ValidData.dataset.__len__(),(2*args.n_hms*args.n_bins))
        all_attention_hm=torch.zeros(ValidData.dataset.__len__(),2*args.n_hms)
    elif(args.model_name=='raw'):
        all_attention_bin=torch.zeros(ValidData.dataset.__len__(),(3*args.n_hms*args.n_bins))
        all_attention_hm=torch.zeros(ValidData.dataset.__len__(),3*args.n_hms)
    elif(args.model_name=='aux' or args.model_name=='aux_siamese'):
        all_attention_bin=torch.zeros(ValidData.dataset.__len__(),(2*args.n_hms*args.n_bins))
        all_attention_hm=torch.zeros(ValidData.dataset.__len__(),2*args.n_hms)
    elif(args.model_name=='raw_aux' or args.model_name=='raw_aux_siamese'):
        all_attention_bin=torch.zeros(ValidData.dataset.__len__(),(3*args.n_hms*args.n_bins))
        all_attention_hm=torch.zeros(ValidData.dataset.__len__(),5*args.n_hms)
    else:
        all_attention_bin=torch.zeros(ValidData.dataset.__len__(),(args.n_hms*args.n_bins))
        all_attention_hm=torch.zeros(ValidData.dataset.__len__(),args.n_hms)

    num_batches = int(math.ceil(ValidData.dataset.__len__()/float(args.batch_size)))
    all_gene_ids=[None]*ValidData.dataset.__len__()
    per_epoch_loss = 0
    for idx, Sample in enumerate(ValidData):
        if(idx%100==0):
            print('TESTING ON BATCH:',idx)
        start,end = (idx*args.batch_size), min((idx*args.batch_size)+args.batch_size, ValidData.dataset.__len__())
        optimizer.zero_grad()
        # get HM profiles
        inputs_1 = Sample['X_A']
        inputs_2 = Sample['X_B']

        
        # get targets: both differential and cell specific expression
        batch_diff_targets=(Sample['diff']).float().unsqueeze(1)
        batch_diff_targets_c1=(Sample['abs_A']).float().unsqueeze(1)
        batch_diff_targets_c2=(Sample['abs_B']).float().unsqueeze(1)
        diff_targets[start:end,0] = batch_diff_targets[:,0]

        if(CON==True):
            # get labels for contrastive loss
            batch_contrastive_targets =[]
            for label in batch_diff_targets:
                if(label<=-2.0):
                    batch_contrastive_targets.append(1)
                elif(label>=2.0):
                    batch_contrastive_targets.append(1)
                else:
                    batch_contrastive_targets.append(0)
            batch_contrastive_targets=torch.Tensor(batch_contrastive_targets)


        all_gene_ids[start:end]=Sample['geneID']
        batch_size = inputs_1.size(0)

        if(AUX==False):
            batch_diff_predictions,batch_beta,batch_alpha = model(inputs_1.type(dtype),inputs_2.type(dtype))

            all_attention_bin[start:end]=batch_alpha.data
            all_attention_hm[start:end]=batch_beta.data
            loss = DiffLoss(batch_diff_predictions,batch_diff_targets.type(dtype))
        elif(CON==False):
            # for aux models
            batch_diff_predictions,batch_beta,batch_alpha,batch_diff_predictions_c1,batch_diff_predictions_c2 = model(inputs_1.type(dtype),inputs_2.type(dtype))
            all_attention_bin[start:end]=batch_alpha.data
            all_attention_hm[start:end]=batch_beta.data
            loss = DiffLoss(batch_diff_predictions,batch_diff_targets.type(dtype))
            loss+=AuxLoss(batch_diff_predictions_c1,batch_diff_targets_c1.type(dtype))
            loss+=AuxLoss(batch_diff_predictions_c2,batch_diff_targets_c2.type(dtype))
        else:
            # for aux and siamese models
            batch_diff_predictions,batch_beta,batch_alpha,embedding_1,embedding_2,batch_diff_predictions_c1,batch_diff_predictions_c2 = model(inputs_1.type(dtype),inputs_2.type(dtype))

            all_attention_bin[start:end]=batch_alpha.data
            all_attention_hm[start:end]=batch_beta.data
            loss = DiffLoss(batch_diff_predictions,batch_diff_targets.type(dtype))
            loss+=AuxLoss(batch_diff_predictions_c1,batch_diff_targets_c1.type(dtype))
            loss+=AuxLoss(batch_diff_predictions_c2,batch_diff_targets_c2.type(dtype))
            loss+=args.gamma*ConLoss(embedding_1,embedding_2,batch_contrastive_targets.type(dtype))

        diff_predictions[start:end] = batch_diff_predictions.data.cpu()
        per_epoch_loss += loss.item()
    per_epoch_loss=per_epoch_loss/num_batches
    return diff_predictions,diff_targets,all_attention_bin,all_attention_hm,per_epoch_loss,all_gene_ids



    # Train = torch.utils.data.DataLoader(train_inputs, batch_size=args.batch_size, shuffle=True)
    # Valid = torch.utils.data.DataLoader(valid_inputs, batch_size=args.batch_size, shuffle=False)
    # Test = torch.utils.data.DataLoader(test_inputs, batch_size=args.batch_size, shuffle=False)



best_valid_loss = 10000000000
best_valid_MSE=100000
best_valid_R2=-1
if(args.test_on_saved_model==False):
    for epoch in range(0, args.epochs):
        print('=---------------------------------------- Training '+str(epoch+1)+' -----------------------------------=')
        diff_predictions,diff_targets,alpha_train,beta_train,train_loss,_ = train(Train)
        train_MSE, train_R2 = evaluate.compute_metrics(diff_predictions,diff_targets)
        diff_predictions,diff_targets,alpha_valid,beta_valid,valid_loss,gene_ids_valid = test(Valid)
        valid_MSE, valid_R2 = evaluate.compute_metrics(diff_predictions,diff_targets)

        if(valid_R2 >= best_valid_R2):
                # save best epoch -- models converge early
            best_valid_R2=valid_R2
            torch.save(model,model_dir+"/"+model_name+'_R2_model.pt')

        print("Epoch:",epoch)
        print("train R2:",train_R2)
        print("valid R2:",valid_R2)
        print("best valid R2:", best_valid_R2)

 
    print("finished training!!")
    print("best validation R2:",best_valid_R2)
    print("testing")
    model=torch.load(model_dir+"/"+model_name+'_R2_model.pt')

    diff_predictions,diff_targets,alpha_test,beta_test,test_loss,gene_ids_test = test(Test)
    test_MSE, test_R2 = evaluate.compute_metrics(diff_predictions,diff_targets)
    print("test R2:",test_R2)

    if(args.save_attention_maps):
        attentionfile=open(attentionmapfile,'w')
        attentionfilewriter=csv.writer(attentionfile)
        beta_test=beta_test.numpy()
        for i in range(len(gene_ids_test)):
            gene_attention=[]
            gene_attention.append(gene_ids_test[i])
            for e in beta_test[i,:]:
                gene_attention.append(str(e))
            attentionfilewriter.writerow(gene_attention)
        attentionfile.close()


else:
    model=torch.load(model_dir+"/"+model_name+'_R2_model.pt')
    diff_predictions,diff_targets,alpha_test,beta_test,test_loss,gene_ids_test = test(Test)
    test_MSE, test_R2 = evaluate.compute_metrics(diff_predictions,diff_targets)
    print("test R2:",test_R2)

    if(args.save_attention_maps):
        attentionfile=open(attentionmapfile,'w')
        attentionfilewriter=csv.writer(attentionfile)
        beta_test=beta_test.numpy()
        for i in range(len(gene_ids_test)):
            gene_attention=[]
            gene_attention.append(gene_ids_test[i])
            for e in beta_test[i,:]:
                gene_attention.append(str(e))
            attentionfilewriter.writerow(gene_attention)
        attentionfile.close()


In [0]:
import torch
import collections
import pdb
import torch.utils.data
import csv
import json
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import math
import numpy as np

def getlabel(c1,c2):
    # get log fold change of expression

    label1=math.log((float(c1)+1.0),2)
    label2=math.log((float(c2)+1.0),2)
    label=[]
    label.append(label1)
    label.append(label2)

    fold_change=(float(c2)+1.0)/(float(c1)+1.0)
    log_fold_change=math.log((fold_change),2)
    return (log_fold_change, label)

def loadDict(filename):
    # get expression value of each gene from cell*.expr.csv
    gene_dict={}
    with open(filename) as fi:
        for line in fi:
            geneID,geneExpr=line.split(',')
            gene_dict[str(geneID)]=float(geneExpr)
    fi.close()
    return(gene_dict)


def loadData(filename,windows,gene_dict, num_hms):
    
    with open(filename) as fi:
        csv_reader=csv.reader(fi)
        data = list(csv_reader)
    
        ncols=(len(data[0]))
    fi.close()

    nrows=len(data)
    ngenes=nrows/windows
    nfeatures=ncols-1
    
    # testing args.n_hms vs nfeatures
    if nfeatures != num_hms:
        print("nfeatures != num_hms")
    assert(int(nfeatures) == int(num_hms))

    
    print("Number of genes: %d" % ngenes)
    print("Number of entries: %d" % nrows)
    print("Number of HMs: %d" % nfeatures)

    count = 0
    attr = collections.OrderedDict()
    alph = [chr(x) for x in range(ord('a'), ord('z') + 1)] # alphabetical key order more stable

    gene_keys = ["geneID", "expr"]
    hm_id = ["hm_" + alph[i] for i in list(range(nfeatures))]
    data_mat = np.array(data)

    for i in range(0, nrows, windows):
        geneID=str(data[i][0].split("_")[0])
    
        meta = {}
        meta["geneID"] = geneID
        meta["expr"] = gene_dict[geneID]
    
        for j in range(nfeatures):
            meta[hm_id[j]] = torch.tensor(np.array([float(z) for z in data_mat[i:(i+windows), j+1]]).reshape(windows, 1))
    
        attr[count] = meta
        count+=1    
        
    return attr

#### NEED TO EDIT THIS TO TAKE N-FEATURES

class HMData(Dataset):
    # Dataset class for loading data
    def __init__(self,data_cell1,data_cell2, n_feat, transform = None): 
        self.c1=data_cell1
        self.c2=data_cell2
        self.nfeat = n_feat
        assert (len(self.c1)==len(self.c2))
    def __len__(self):
        return len(self.c1)
    def __getitem__(self,i):
                 
        alph = [chr(x) for x in range(ord('a'), ord('z') + 1)]
        hm_id = ["hm_" + alph[y] for y in list(range(self.nfeat))]
                 
#         final_data_c1=torch.cat((self.c1[i]['hm1'],self.c1[i]['hm2'],self.c1[i]['hm3'],self.c1[i]['hm4'],self.c1[i]['hm5']),1)
#         final_data_c2=torch.cat((self.c2[i]['hm1'],self.c2[i]['hm2'],self.c2[i]['hm3'],self.c2[i]['hm4'],self.c2[i]['hm5']),1)
        
        final_data_c1 = torch.cat([self.c1[i][z] for z in hm_id], 1) 
        final_data_c2 = torch.cat([self.c2[i][w] for w in hm_id], 1)         
                
        label,orig_label=getlabel(self.c1[i]['expr'],self.c2[i]['expr'])
        b_label_c1=orig_label[0]
        b_label_c2=orig_label[1]
        assert self.c1[i]['geneID']==self.c2[i]['geneID']
        geneID=self.c1[i]['geneID']
        sample={'geneID':geneID,
               'X_A':final_data_c1,
               'X_B':final_data_c2,
               'diff':label,
               'abs_A':b_label_c1,'abs_B':b_label_c2}
        return sample
    
### NEED TO PASS args.n_hms TO HMDATA FOR nfeat ARGUMENT    
#  assert that nfeatures == args.n_hms
#  could also just pass nfeatures from loadData (still needs assert)

def load_data(args):
    '''
    Loads data into a 3D tensor for each of the 3 splits.

    '''
    print("==>loading train data")
    gene_dict1=loadDict(args.data_root+args.cell_1+".expr.csv")
    gene_dict2=loadDict(args.data_root+args.cell_2+".expr.csv")
    
    cell_train_dict1=loadData(args.data_root+"/"+args.cell_1+".train.csv",
                              args.n_bins,gene_dict1, args.n_hms) # n_hms assert    
    cell_train_dict2=loadData(args.data_root+"/"+args.cell_2+".train.csv",
                              args.n_bins,gene_dict2, args.n_hms) # n_hms assert
    train_inputs = HMData(cell_train_dict1,cell_train_dict2, args.n_hms) # added dynamic args.n_hms 
    
    print("==>loading valid data")
    cell_valid_dict1=loadData(args.data_root+"/"+args.cell_1+".valid.csv", 
                              args.n_bins,gene_dict1, args.n_hms) 
    cell_valid_dict2=loadData(args.data_root+"/"+args.cell_2+".valid.csv", 
                              args.n_bins,gene_dict2, args.n_hms)   
    valid_inputs = HMData(cell_valid_dict1,cell_valid_dict2, args.n_hms) 
    
    print("==>loading test data")
    cell_test_dict1=loadData(args.data_root+"/"+args.cell_1+".test.csv", 
                             args.n_bins,gene_dict1, args.n_hms) 
    cell_test_dict2=loadData(args.data_root+"/"+args.cell_2+".test.csv", 
                             args.n_bins,gene_dict2, args.n_hms)   
    test_inputs = HMData(cell_test_dict1,cell_test_dict2, args.n_hms)

      
    return train_inputs, valid_inputs, test_inputs # return inputs to trainloading/cross validation

In [0]:

def batch_product(iput, mat2):
        result = None
        for i in range(iput.size()[0]):
            op = torch.mm(iput[i], mat2)
            op = op.unsqueeze(0)
            if(result is None):
                result = op
            else:
                result = torch.cat((result,op),0)
        return result.squeeze(2)


class rec_attention(nn.Module):
    # attention with bin context vector per HM and HM context vector
    def __init__(self,hm,args):
        super(rec_attention,self).__init__()
        self.num_directions=2 if args.bidirectional else 1
        if (hm==False):
            self.bin_rep_size=args.bin_rnn_size*self.num_directions
        else:
            self.bin_rep_size=args.bin_rnn_size
    
        self.bin_context_vector=nn.Parameter(torch.Tensor(self.bin_rep_size,1),requires_grad=True)
    

        self.softmax=nn.Softmax()

        self.bin_context_vector.data.uniform_(-0.1, 0.1)

    def forward(self,iput):
        alpha=self.softmax(batch_product(iput,self.bin_context_vector))
        [batch_size,source_length,bin_rep_size2]=iput.size()
        repres=torch.bmm(alpha.unsqueeze(2).view(batch_size,-1,source_length),iput)
        return repres,alpha



class recurrent_encoder(nn.Module):
    # modular LSTM encoder
    def __init__(self,n_bins,ip_bin_size,hm,args):
        super(recurrent_encoder,self).__init__()
        self.bin_rnn_size=args.bin_rnn_size
        self.ipsize=ip_bin_size
        self.seq_length=n_bins

        self.num_directions=2 if args.bidirectional else 1
        if (hm==False):
            self.bin_rnn_size=args.bin_rnn_size
        else:
            self.bin_rnn_size=args.bin_rnn_size // 2
        self.bin_rep_size=self.bin_rnn_size*self.num_directions


        self.rnn=nn.LSTM(self.ipsize,self.bin_rnn_size,num_layers=args.num_layers,dropout=args.dropout,bidirectional=args.bidirectional)

        self.bin_attention=rec_attention(hm,args)
    def outputlength(self):
        return self.bin_rep_size
    def forward(self,single_hm,hidden=None):

        bin_output, hidden = self.rnn(single_hm,hidden)
        bin_output = bin_output.permute(1,0,2)
        hm_rep,bin_alpha = self.bin_attention(bin_output)
        return hm_rep,bin_alpha

class raw_d(nn.Module):
    def __init__(self,args):
        self.n_hms=args.n_hms
        self.n_bins=args.n_bins
        self.ip_bin_size=1
        super(raw_d,self).__init__()
        self.rnn_hms=nn.ModuleList()
        for i in range(self.n_hms):
            self.rnn_hms.append(recurrent_encoder(self.n_bins,self.ip_bin_size,False,args))
        self.opsize = self.rnn_hms[0].outputlength()
        self.hm_level_rnn_1=recurrent_encoder(self.n_hms,self.opsize,True,args)
        self.opsize2=self.hm_level_rnn_1.outputlength()
        self.diffopsize=2*(self.opsize2)
        self.fdiff1_1=nn.Linear(self.opsize2,1)

    def forward(self,iput1,iput2):

        iput=iput1-iput2
        bin_a=None
        level1_rep=None
        [batch_size,_,_]=iput.size()

        for hm,hm_encdr in enumerate(self.rnn_hms):
            hmod=iput[:,:,hm].contiguous()
            hmod=torch.t(hmod).unsqueeze(2)

            op,a= hm_encdr(hmod)
            if level1_rep is None:
                level1_rep=op
                bin_a=a
            else:
                level1_rep=torch.cat((level1_rep,op),1)
                bin_a=torch.cat((bin_a,a),1)

        #--------TARGET DIMENSION FIX--------No idea if this is right
        # flatten bin_a to match all_attention dimensions
        # print("Orig: ", bin_a.shape)
        # bin_a = bin_a.reshape(-1)
        # print("Reshape: ", bin_a.shape)

        
        level1_rep=level1_rep.permute(1,0,2)
        final_rep_1,hm_level_attention_1=self.hm_level_rnn_1(level1_rep)
        final_rep_1=final_rep_1.squeeze(1)
        prediction_m=((self.fdiff1_1(final_rep_1)))
        return prediction_m,hm_level_attention_1, bin_a


class raw_c(nn.Module):
    def __init__(self,args):
        super(raw_c,self).__init__()
        self.n_hms=args.n_hms
        self.n_bins=args.n_bins
        self.ip_bin_size=1
        self.joint=False
        self.rnn_hms=nn.ModuleList()
        for i in range(2*self.n_hms):
            self.rnn_hms.append(recurrent_encoder(self.n_bins,self.ip_bin_size,False,args))
        self.opsize = self.rnn_hms[0].outputlength()
        self.hm_level_rnn_1=recurrent_encoder(2*self.n_hms,self.opsize,True,args)
        self.opsize2=self.hm_level_rnn_1.outputlength()
        self.diffopsize=2*(self.opsize2)
        self.fdiff1_1=nn.Linear(self.opsize2,1)

    def forward(self,iput1,iput2):
        iput=torch.cat((iput1,iput2),2)
        bin_a=None
        level1_rep=None
        for hm,hm_encdr in enumerate(self.rnn_hms):
            hmod=iput[:,:,hm].contiguous()

            hmod=torch.t(hmod).unsqueeze(2)
            op,a= hm_encdr(hmod)

            if level1_rep is None:
                level1_rep=op
                bin_a=a
            else:
                level1_rep=torch.cat((level1_rep,op),1)
                bin_a=torch.cat((bin_a,a),1)

        #--------TARGET DIMENSION FIX--------No idea if this is right
        # flatten bin_a to match all_attention dimensions
        # bin_a = bin_a.reshape(-1)

        level1_rep=level1_rep.permute(1,0,2)
        final_rep_1,hm_level_attention_1=self.hm_level_rnn_1(level1_rep)
        final_rep_1=final_rep_1.squeeze(1)
        prediction_m=((self.fdiff1_1(final_rep_1)))

        return prediction_m,hm_level_attention_1, bin_a


class raw(nn.Module):
    # Model with all raw features: difference and absolute features
    def __init__(self,args):
        super(raw,self).__init__()
        self.n_hms=args.n_hms
        self.n_bins=args.n_bins
        self.ip_bin_size=1
        self.rnn_hms=nn.ModuleList()
        for i in range(3*self.n_hms):
            self.rnn_hms.append(recurrent_encoder(self.n_bins,self.ip_bin_size,False,args))
        self.opsize = self.rnn_hms[0].outputlength()
        self.hm_level_rnn_1=recurrent_encoder(3*self.n_hms,self.opsize,True,args)
        self.opsize2=self.hm_level_rnn_1.outputlength()
        self.diffopsize=2*(self.opsize2)
        self.fdiff1_1=nn.Linear(self.opsize2,1)

    def forward(self,iput1,iput2):
        iput3=iput1-iput2
        iput4=torch.cat((iput1,iput2),2)
        iput=(torch.cat((iput4,iput3),2))

        bin_a=None
        level1_rep=None
        for hm,hm_encdr in enumerate(self.rnn_hms):
            hmod=iput[:,:,hm].contiguous()
            hmod=torch.t(hmod).unsqueeze(2)
            op,a= hm_encdr(hmod)
            if level1_rep is None:
                level1_rep=op
                bin_a=a
            else:
                level1_rep=torch.cat((level1_rep,op),1)
                bin_a=torch.cat((bin_a,a),1)

        #--------TARGET DIMENSION FIX--------No idea if this is right
        # flatten bin_a to match all_attention dimensions
        # bin_a = bin_a.reshape(-1)

        level1_rep=level1_rep.permute(1,0,2)
        final_rep_1,hm_level_attention_1=self.hm_level_rnn_1(level1_rep)
        final_rep_1=final_rep_1.squeeze(1)

        prediction_m=((self.fdiff1_1(final_rep_1)))
        return prediction_m,hm_level_attention_1, bin_a


class aux(nn.Module):
    def __init__(self,args):
        super(aux,self).__init__()
        self.n_hms=args.n_hms
        self.n_bins=args.n_bins
        self.ip_bin_size=1
        self.joint=False
        self.shared=False
        self.rnn_hms=nn.ModuleList()
        for i in range(self.n_hms):
            self.rnn_hms.append(recurrent_encoder(self.n_bins,self.ip_bin_size,False,args))
        self.opsize = self.rnn_hms[0].outputlength()
        self.rnn_hms2=nn.ModuleList()
        for i in range(self.n_hms):
            self.rnn_hms2.append(recurrent_encoder(self.n_bins,self.ip_bin_size,False,args))
        self.hm_level_rnn_1=recurrent_encoder(self.n_hms,self.opsize,True,args)
        self.hm_level_rnn_2=recurrent_encoder(self.n_hms,self.opsize,True,args)
        self.opsize2=self.hm_level_rnn_1.outputlength()
        self.f1_1=nn.Linear(self.opsize2,1)
        self.f2_1=nn.Linear(self.opsize2,1)
        self.diffopsize=2*(self.opsize2)
        self.predictor_1=nn.Linear(self.diffopsize,self.diffopsize//2)
        self.predictor_=nn.Linear(self.diffopsize//2,1)
        self.relu=nn.ReLU()

    def forward_once(self,iput,shared,cellid):
        bin_a=None
        level1_rep=None
        if(shared or cellid==1):
            for hm,hm_encdr in enumerate(self.rnn_hms):
                hmod=iput[:,:,hm].contiguous()
                hmod=torch.t(hmod).unsqueeze(2)
                op,a= hm_encdr(hmod)
                if level1_rep is None:
                    level1_rep=op
                    bin_a=a
                else:
                    level1_rep=torch.cat((level1_rep,op),1)
                    bin_a=torch.cat((bin_a,a),1)

            #--------TARGET DIMENSION FIX--------No idea if this is right
            # flatten bin_a to match all_attention dimensions
            # bin_a = bin_a.reshape(-1)

            level1_rep=level1_rep.permute(1,0,2)
        else:
            for hm,hm_encdr in enumerate(self.rnn_hms2):

                hmod=iput[:,:,hm].contiguous()
                hmod=torch.t(hmod).unsqueeze(2)
                op,a= hm_encdr(hmod)
                if level1_rep is None:
                    level1_rep=op
                    bin_a=a
                else:
                    level1_rep=torch.cat((level1_rep,op),1)
                    bin_a=torch.cat((bin_a,a),1)
 
            #--------TARGET DIMENSION FIX--------No idea if this is right
            # flatten bin_a to match all_attention dimensions
            # bin_a = bin_a.reshape(-1)

            level1_rep=level1_rep.permute(1,0,2)
        return level1_rep,bin_a

    def forward(self,iput1,iput2):
 
        level1_rep1,bin_a1=self.forward_once(iput1,self.shared,1)
        level1_rep2,bin_a2=self.forward_once(iput2,self.shared,2)
        final_rep_1,hm_level_attention_1=self.hm_level_rnn_1(level1_rep1)
        if(self.joint):
            final_rep_2,hm_level_attention_2=self.hm_level_rnn_1(level1_rep2)
        else:
            final_rep_2,hm_level_attention_2=self.hm_level_rnn_2(level1_rep2)
        final_rep_1=final_rep_1.squeeze(1)
        final_rep_2=final_rep_2.squeeze(1)
        prediction1=((self.f1_1(final_rep_1)))
        prediction2=((self.f2_1(final_rep_2)))

        mlp_input=torch.cat((final_rep_1,final_rep_2),1)
        mlp_input=self.relu(self.predictor_1(mlp_input))
        prediction=(self.predictor_(mlp_input))
        hm_level_attention=torch.cat((hm_level_attention_1,hm_level_attention_2),1)
        
        bin_a=torch.cat((bin_a1,bin_a2),1)
        # already flattened bin_a's
        # bin_a = torch.cat((bin_a1, bin_a2))


        return prediction,hm_level_attention,bin_a,prediction1,prediction2


class raw_aux(nn.Module):
    # level 1 takes raw feaures, level 2 takes aux features as well as raw feature embeddings from level 1
    # returns attention scores from raw part only
    def __init__(self,args):
        super(raw_aux,self).__init__()
        self.n_hms=args.n_hms
        self.n_bins=args.n_bins
        self.ip_bin_size=1
        self.joint=False
        self.shared=False
        self.rnn_hms=nn.ModuleList()
        for i in range(self.n_hms):
            self.rnn_hms.append(recurrent_encoder(self.n_bins,self.ip_bin_size,False,args))
        self.opsize = self.rnn_hms[0].outputlength()
        self.rnn_hms3=nn.ModuleList()
        for i in range(3*self.n_hms):
            self.rnn_hms3.append(recurrent_encoder(self.n_bins,self.ip_bin_size,False,args))
        self.opsize = self.rnn_hms[0].outputlength()
        self.rnn_hms2=nn.ModuleList()
        for i in range(self.n_hms):
            self.rnn_hms2.append(recurrent_encoder(self.n_bins,self.ip_bin_size,False,args))
        self.hm_level_rnn_1=recurrent_encoder(self.n_hms,self.opsize,True,args)
        self.hm_level_rnn_3=recurrent_encoder(5*self.n_hms,self.opsize,True,args)
        self.hm_level_rnn_2=recurrent_encoder(self.n_hms,self.opsize,True,args)
        self.opsize2=self.hm_level_rnn_1.outputlength()
        self.f1_1=nn.Linear(self.opsize2,1)
        self.f2_1=nn.Linear(self.opsize2,1)
        self.diffopsize=3*(self.opsize2)
        self.predictor_=nn.Linear(self.opsize2,1)
        self.relu=nn.ReLU()


    def forward(self,iput1,iput2):
        iput3=iput1-iput2
        iput4=torch.cat((iput1,iput2),2)
        iput=(torch.cat((iput4,iput3),2))
        bin_a=None
        level1_rep=None
        [batch_size,_,_]=iput.size()
        for hm,hm_encdr in enumerate(self.rnn_hms3):

            hmod=iput[:,:,hm].contiguous()
            hmod=torch.t(hmod).unsqueeze(2)
            op,a= hm_encdr(hmod)
            if level1_rep is None:
                level1_rep=op
                bin_a=a
            else:
                level1_rep=torch.cat((level1_rep,op),1)
                bin_a=torch.cat((bin_a,a),1)

        #--------TARGET DIMENSION FIX--------No idea if this is right
        # flatten bin_a to match all_attention dimensions
        # bin_a = bin_a.reshape(-1)

        level1_rep=level1_rep.permute(1,0,2)

        bin_a1=None
        level1_rep1=None
        [batch_size,_,_]=iput1.size()
        for hm,hm_encdr in enumerate(self.rnn_hms):
            hmod=iput1[:,:,hm].contiguous()
            hmod=torch.t(hmod).unsqueeze(2)
            op,a= hm_encdr(hmod)
            if level1_rep1 is None:
                level1_rep1=op
                bin_a1=a
            else:
                level1_rep1=torch.cat((level1_rep1,op),1)
                bin_a1=torch.cat((bin_a1,a),1)

        #--------TARGET DIMENSION FIX--------No idea if this is right
        # flatten bin_a to match all_attention dimensions
        # bin_a1 = bin_a1.reshape(-1)

        level1_rep1=level1_rep1.permute(1,0,2)
        bin_a2=None
        level1_rep2=None

        for hm,hm_encdr in enumerate(self.rnn_hms2):
            hmod=iput2[:,:,hm].contiguous()
            hmod=torch.t(hmod).unsqueeze(2)
            op,a= hm_encdr(hmod)
            if level1_rep2 is None:
                level1_rep2=op
                bin_a2=a
            else:
                level1_rep2=torch.cat((level1_rep2,op),1)
                bin_a2=torch.cat((bin_a2,a),1)

        #--------TARGET DIMENSION FIX--------No idea if this is right
        # flatten bin_a to match all_attention dimensions
        # bin_a2 = bin_a2.reshape(-1)

        level1_rep2=level1_rep2.permute(1,0,2)
        level1_rep3=torch.cat((level1_rep,level1_rep1,level1_rep2),0)
        final_rep_3,hm_level_attention_3=self.hm_level_rnn_3(level1_rep3)
        final_rep_3=final_rep_3.squeeze(1)

        final_rep_1,hm_level_attention_1=self.hm_level_rnn_1(level1_rep1)
        final_rep_2,hm_level_attention_2=self.hm_level_rnn_2(level1_rep2)
        final_rep_1=final_rep_1.squeeze(1)
        final_rep_2=final_rep_2.squeeze(1)
        prediction1=((self.f1_1(final_rep_1)))
        prediction2=((self.f2_1(final_rep_2)))
        prediction=(self.predictor_(final_rep_3))

        # bin_a1 and bin_a2 supposed to be concatenated like aux and aux_siamese????

        return prediction,hm_level_attention_3,bin_a,prediction1,prediction2


class aux_siamese(nn.Module):
    # aux with siamese  same as aux model mostly, but with shared level 1 embedding
    def __init__(self,args):
        super(aux_siamese,self).__init__()
        self.n_hms=args.n_hms
        self.n_bins=args.n_bins
        self.ip_bin_size=1
        self.joint=False
        self.shared=True
        self.rnn_hms=nn.ModuleList()
        for i in range(self.n_hms):
            self.rnn_hms.append(recurrent_encoder(self.n_bins,self.ip_bin_size,False,args))
        self.opsize = self.rnn_hms[0].outputlength()
        self.hm_level_rnn_1=recurrent_encoder(self.n_hms,self.opsize,True,args)
        self.hm_level_rnn_2=recurrent_encoder(self.n_hms,self.opsize,True,args)
        self.opsize2=self.hm_level_rnn_1.outputlength()
        self.f1_1=nn.Linear(self.opsize2,1)
        self.f2_1=nn.Linear(self.opsize2,1)
        self.diffopsize=2*(self.opsize2)
        self.predictor_1=nn.Linear(self.diffopsize,self.diffopsize//2)
        self.predictor_=nn.Linear(self.diffopsize//2,1)
        self.relu=nn.ReLU()

    def forward_once(self,iput):
        bin_a=None
        level1_rep=None
        [batch_size,_,_]=iput.size()
        for hm,hm_encdr in enumerate(self.rnn_hms):
            hmod=iput[:,:,hm].contiguous()
            hmod=torch.t(hmod).unsqueeze(2)
            op,a= hm_encdr(hmod)
            if level1_rep is None:
                level1_rep=op
                bin_a=a
            else:
                level1_rep=torch.cat((level1_rep,op),1)
                bin_a=torch.cat((bin_a,a),1)
        
        #--------TARGET DIMENSION FIX--------No idea if this is right
        # flatten bin_a to match all_attention dimensions
        # bin_a = bin_a.reshape(-1)

        level1_rep=level1_rep.permute(1,0,2)

        return level1_rep,bin_a


    def forward(self,iput1,iput2):
        level1_rep1,bin_a1=self.forward_once(iput1)

        level1_rep2,bin_a2=self.forward_once(iput2)
        final_rep_1,hm_level_attention_1=self.hm_level_rnn_1(level1_rep1)
        final_rep_2,hm_level_attention_2=self.hm_level_rnn_2(level1_rep2)
        final_rep_1=final_rep_1.squeeze(1)
        final_rep_2=final_rep_2.squeeze(1)
        [a,b,c]=(level1_rep1.size())
        alpha_rep_1=level1_rep1.permute(1,0,2).view(b,a*c)
        alpha_rep_2=level1_rep2.permute(1,0,2).view(b,a*c)
        prediction1=((self.f1_1(final_rep_1)))
        prediction2=((self.f2_1(final_rep_2)))
        mlp_input=torch.cat((final_rep_1,final_rep_2),1)
        mlp_input=self.relu(self.predictor_1(mlp_input))
        prediction=(self.predictor_(mlp_input))
        hm_level_attention=torch.cat((hm_level_attention_1,hm_level_attention_2),1)
        
        bin_a=torch.cat((bin_a1,bin_a2),1)
        # already flattened bin_a's
        # bin_a = torch.cat((bin_a1, bin_a2))

        return prediction,hm_level_attention,bin_a,alpha_rep_1,alpha_rep_2,prediction1,prediction2


class raw_aux_siamese(nn.Module):
    # similar to raw_aux model with shared level 1 embedding
    # returns only raw level attentions
    # returns embeddings from shared level 1 for contrastive loss
    def __init__(self,args):
        self.n_hms=args.n_hms
        self.n_bins=args.n_bins
        self.ip_bin_size=1
        self.joint=False
        self.shared=True
        super(raw_aux_siamese,self).__init__()
        self.rnn_hms=nn.ModuleList()
        self.rnn_hmsx=nn.ModuleList()
        for i in range(self.n_hms):
            self.rnn_hms.append(recurrent_encoder(self.n_bins,self.ip_bin_size,False,args))
        for i in range(3*self.n_hms):
            self.rnn_hmsx.append(recurrent_encoder(self.n_bins,self.ip_bin_size,False,args))
        self.opsize = self.rnn_hms[0].outputlength()
        self.hm_level_rnn_1=recurrent_encoder(self.n_hms,self.opsize,True,args)
        self.hm_level_rnn_1x=recurrent_encoder(5*self.n_hms,self.opsize,True,args)
        self.hm_level_rnn_2=recurrent_encoder(self.n_hms,self.opsize,True,args)
        self.opsize2=self.hm_level_rnn_1.outputlength()
        self.f1_1=nn.Linear(self.opsize2,1)
        self.f2_1=nn.Linear(self.opsize2,1)
        self.diffopsize=3*(self.opsize2)
        #self.predictor_1=nn.Linear(self.diffopsize,self.diffopsize//2)
        self.predictor_=nn.Linear(self.opsize2,1)
        self.relu=nn.ReLU()
        self.finalsoftmax=nn.LogSoftmax()
    def forward_once(self,iput):
        bin_a=None
        level1_rep=None
        [batch_size,_,_]=iput.size()
        for hm,hm_encdr in enumerate(self.rnn_hms):
            hmod=iput[:,:,hm].contiguous()
            hmod=torch.t(hmod).unsqueeze(2)
            op,a= hm_encdr(hmod)
            if level1_rep is None:
                level1_rep=op
                bin_a=a
            else:
                level1_rep=torch.cat((level1_rep,op),1)
                bin_a=torch.cat((bin_a,a),1)
        level1_rep=level1_rep.permute(1,0,2)
        return level1_rep,bin_a


    def forward(self,iput1,iput2):
        iput3=iput1-iput2
        iput4=torch.cat((iput1,iput2),2)
        iput=(torch.cat((iput4,iput3),2))
        bin_ax=None
        level1_repx=None
        [batch_size,_,_]=iput.size()
        for hm,hm_encdr in enumerate(self.rnn_hmsx):
            hmodx=iput[:,:,hm].contiguous()
            hmodx=torch.t(hmodx).unsqueeze(2)
            opx,ax= hm_encdr(hmodx)
            if level1_repx is None:
                level1_repx=opx
                bin_ax=ax
            else:
                level1_repx=torch.cat((level1_repx,opx),1)
                bin_ax=torch.cat((bin_ax,ax),1)
        level1_repx=level1_repx.permute(1,0,2)

        level1_rep1,bin_a1=self.forward_once(iput1)

        level1_rep2,bin_a2=self.forward_once(iput2)
        final_rep_1,hm_level_attention_1=self.hm_level_rnn_1(level1_rep1)
        final_rep_2,hm_level_attention_2=self.hm_level_rnn_2(level1_rep2)
        final_rep_1=final_rep_1.squeeze(1)
        final_rep_2=final_rep_2.squeeze(1)
        [a,b,c]=(level1_rep1.size())
        alpha_rep_1=level1_rep1.permute(1,0,2).view(b,a*c)
        alpha_rep_2=level1_rep2.permute(1,0,2).view(b,a*c)
        level1_rep3=torch.cat((level1_repx,level1_rep1,level1_rep2),0)
        final_rep_1x,hm_level_attention_1x=self.hm_level_rnn_1x(level1_rep3)
        final_rep_1x=final_rep_1x.squeeze(1)
        prediction1=((self.f1_1(final_rep_1)))
        prediction2=((self.f2_1(final_rep_2)))
        prediction=(self.predictor_(final_rep_1x))

        return prediction,hm_level_attention_1x,bin_ax,alpha_rep_1,alpha_rep_2,prediction1,prediction2